In [1]:
#on importe toutes les bibliothèques
import pandas as pd

In [2]:
train_data=pd.read_csv("data/train.csv")
X_test=pd.read_csv("data/test.csv")

In [22]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  group         8693 non-null   object 
 15  number        8693 non-null   object 
dtypes: bool(1), float64(6), object(9)
memory usage: 1.0+ MB


In [14]:
#on créer une feature qui représente le groupe et une leur numéro du groupe
train_data["group"]=train_data["PassengerId"].str.slice(stop=4)
train_data["number"]=train_data["PassengerId"].str.slice(start=5)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [18]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
#pour l'instant on n'utilise pas tout, on se contente pour l'instant du plus simple
num_attribs = ["Age", "RoomService", "FoodCourt", "ShoppingMall","Spa","VRDeck"]
cat_attribs = ["HomePlanet", "CryoSleep", "Destination","VIP"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

clf = Pipeline(
    steps=[("preprocessor", preprocess_pipeline), ("classifier", SVC())]
)

X_train=train_data.copy()
X_train=X_train.drop(columns=["Transported"])
y_train=train_data["Transported"]

In [28]:
from sklearn.model_selection import cross_val_score
dt_scores = cross_val_score(clf, X_train, y_train, cv=5)
dt_scores.mean()

0.785232707182613

In [35]:
clf.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat_encoder',
                

In [36]:
y_pred=clf.predict(X_test)

In [37]:
y_pred_df=pd.DataFrame(y_pred, columns = ['Transported'])
y_pred_df=y_pred_df.set_index(X_test["PassengerId"])

In [38]:
y_pred_df.to_csv("C:/Users/miaou/Documents/UTT/ISI-2/PE_IA_RM/Python_script/spaceship_titanic_competition/data/y_pred.csv")

après avoir fait le test sur les modèles suivants : Random forest classique(0.7873010757740087),Gaussian nb(0.7361125761071896),Logistique regression(0.785460209860964),Plus proches voisins(0.7632582512733335),SVC(0.7889130493762866),Linear SVC(0.7845402732017329),Decision tree(0.7391004181470111), Bagging decison tree(0.7795942405691935)

les modèles les plus parformants sont alors : random forest, logistique regression, svc, linear svc

le modèle le plus performant est le SVC

pour le premier test on obtient un score de 0.78512 d'accuracy avec le random forest quand on le poste sur kaggle

On cherche maintenant à obtimiser la pipeline des données